In [3]:
from email.header import Header
from operator import index
from torch.utils.data import Dataset
import os
import pandas as pd
import numpy as np
import torch
import random
import scipy.spatial
import scipy.io


def create_dataset(
    dataset_folder,
    dataset_name,
    val_size,
    gt,
    horizon,
    delim="\t",
    train=True,
    eval=False,
    verbose=False,
):

    if train == True:
        datasets_list = os.listdir(os.path.join(dataset_folder, dataset_name, "train"))
        full_dt_folder = os.path.join(dataset_folder, dataset_name, "train")
    if train == False and eval == False:
        datasets_list = os.listdir(os.path.join(dataset_folder, dataset_name, "val"))
        full_dt_folder = os.path.join(dataset_folder, dataset_name, "val")
    if train == False and eval == True:
        datasets_list = os.listdir(os.path.join(dataset_folder, dataset_name, "test"))
        full_dt_folder = os.path.join(dataset_folder, dataset_name, "test")

    datasets_list = datasets_list

    data = {}
    data_src = []
    data_trg = []
    data_seq_start = []
    data_frames = []
    data_dt = []
    data_peds = []

    val_src = []
    val_trg = []
    val_seq_start = []
    val_frames = []
    val_dt = []
    val_peds = []
    print("ovdje")
    if verbose:
        print("start loading dataset")
        print("validation set size -> %i" % (val_size))

    for i_dt, dt in enumerate(datasets_list):
        if verbose:
            print("%03i / %03i - loading %s" % (i_dt + 1, len(datasets_list), dt))
        # if i_dt == 0 or i_dt == 1:
        # data = pd.read_csv(os.path.join(full_dt_folder, dt), sep="\t", header=None)
        # print(data.info())
        # print(data.head())
        raw_data = pd.read_csv(
            os.path.join(full_dt_folder, dt),
            delimiter=delim,
            names=[
                "frame",
                "ped",
                "x",
                "y",
                "scale",
                "head_x",
                "head_y",
                "height",
                "l_ear_x",
                "l_ear_y",
                "l_elbow_x",
                "l_elbow_y",
                "l_eye_x",
                "l_eye_y",
                "l_foot_x",
                "l_foot_y",
                "l_hand_x",
                "l_hand_y",
                "l_hip_x",
                "l_hip_y",
                "l_knee_x",
                "l_knee_y",
                "l_shoulder_x",
                "l_shoulder_y",
                "neck_x",
                "neck_y",
                "r_ear_x",
                "r_ear_y",
                "r_elbow_x",
                "r_elbow_y",
                "r_eye_x",
                "r_eye_y",
                "r_foot_x",
                "r_foot_y",
                "r_hand_x",
                "r_hand_y",
                "r_hip_x",
                "r_hip_y",
                "r_knee_x",
                "r_knee_y",
                "r_shoulder_x",
                "r_shoulder_y",
                "threshold",
                "width",
                "R_1",
                "R_2",
                "R_3",
                "T_1",
                "T_2",
                "T_3",
            ],
            usecols=list(range(50)),
            na_values="?",
        )

        print(raw_data.columns)
        raw_data.sort_values(by=["frame", "ped"], inplace=True)

        inp, out, info = get_strided_data_clust(raw_data, gt, horizon, 1)

        dt_frames = info["frames"]
        dt_seq_start = info["seq_start"]
        dt_dataset = np.array([i_dt]).repeat(inp.shape[0])
        dt_peds = info["peds"]

        if val_size > 0 and inp.shape[0] > val_size * 2.5:
            if verbose:
                print("created validation from %s" % (dt))
            k = random.sample(np.arange(inp.shape[0]).tolist(), val_size)
            val_src.append(inp[k, :, :])
            val_trg.append(out[k, :, :])
            val_seq_start.append(dt_seq_start[k, :, :])
            val_frames.append(dt_frames[k, :])
            val_dt.append(dt_dataset[k])
            val_peds.append(dt_peds[k])
            inp = np.delete(inp, k, 0)
            out = np.delete(out, k, 0)
            dt_frames = np.delete(dt_frames, k, 0)
            dt_seq_start = np.delete(dt_seq_start, k, 0)
            dt_dataset = np.delete(dt_dataset, k, 0)
            dt_peds = np.delete(dt_peds, k, 0)
        elif val_size > 0:
            if verbose:
                print(
                    "could not create validation from %s, size -> %i"
                    % (dt, inp.shape[0])
                )

        data_src.append(inp)
        data_trg.append(out)
        data_seq_start.append(dt_seq_start)
        data_frames.append(dt_frames)
        data_dt.append(dt_dataset)
        data_peds.append(dt_peds)

    data["src"] = np.concatenate(data_src, 0)
    data["trg"] = np.concatenate(data_trg, 0)
    data["seq_start"] = np.concatenate(data_seq_start, 0)
    data["frames"] = np.concatenate(data_frames, 0)
    data["dataset"] = np.concatenate(data_dt, 0)
    data["peds"] = np.concatenate(data_peds, 0)
    data["dataset_name"] = datasets_list

    mean = data["src"].mean((0, 1))
    std = data["src"].std((0, 1))

    if val_size > 0:
        data_val = {}
        data_val["src"] = np.concatenate(val_src, 0)
        data_val["trg"] = np.concatenate(val_trg, 0)
        data_val["seq_start"] = np.concatenate(val_seq_start, 0)
        data_val["frames"] = np.concatenate(val_frames, 0)
        data_val["dataset"] = np.concatenate(val_dt, 0)
        data_val["peds"] = np.concatenate(val_peds, 0)

        return IndividualTfDataset(data, "train", mean, std), IndividualTfDataset(
            data_val, "validation", mean, std
        )

    return IndividualTfDataset(data, "train", mean, std), None

    return IndividualTfDataset(data, "train", mean, std), IndividualTfDataset(
        data_val, "validation", mean, std
    )


class IndividualTfDataset(Dataset):
    def __init__(self, data, name, mean, std):
        super(IndividualTfDataset, self).__init__()

        self.data = data
        self.name = name

        self.mean = mean
        self.std = std

    def __len__(self):
        return self.data["src"].shape[0]

    def __getitem__(self, index):
        return {
            "src": torch.Tensor(self.data["src"][index]),
            "trg": torch.Tensor(self.data["trg"][index]),
            "frames": self.data["frames"][index],
            "seq_start": self.data["seq_start"][index],
            "dataset": self.data["dataset"][index],
            "peds": self.data["peds"][index],
        }


def create_folders(baseFolder, datasetName):
    try:
        os.mkdir(baseFolder)
    except:
        pass

    try:
        os.mkdir(os.path.join(baseFolder, datasetName))
    except:
        pass



def get_strided_data_clust(dt, gt_size, horizon, step):
    inp_te = []
    dtt = dt.astype(np.float32)
    raw_data = dtt

    ped = raw_data.ped.unique()
    print("dt columns")
    print(dt.columns)
    frame = []
    ped_ids = []
    # print(dt)
    # print(gt_size)
    # print(horizon)
    # print(step)
    for p in ped:
        # print(f"p is  {p}")
        # print("u p in ped")
        # print(raw_data[raw_data.ped == p].shape)

        # print(1 + (raw_data[raw_data.ped == p].shape[0] - gt_size - horizon) // step)

        for i in range(
            1 + (raw_data[raw_data.ped == p].shape[0] - gt_size - horizon) // step
        ):

            frame.append(
                dt[dt.ped == p]
                .iloc[i * step : i * step + gt_size + horizon, [0]]
                .values.squeeze()
            )
            # print("%i,%i,%i" % (i * 4, i * 4 + gt_size, i * 4 + gt_size + horizon))
            inp_te.append(
                raw_data[raw_data.ped == p]
                .iloc[i * step : i * step + gt_size + horizon, 2:50]
                .values
            )
            ped_ids.append(p)

    frames = np.stack(frame)
    inp_te_np = np.stack(inp_te)
    ped_ids = np.stack(ped_ids)

    # inp_relative_pos= inp_te_np-inp_te_np[:,:1,:]
    inp_speed = np.concatenate(
        (
            np.zeros((inp_te_np.shape[0], 1, 2)),
            inp_te_np[:, 1:, 0:2] - inp_te_np[:, :-1, 0:2],
        ),
        1,
    )
    # inp_accel = np.concatenate((np.zeros((inp_te_np.shape[0],1,2)),inp_speed[:,1:,0:2] - inp_speed[:, :-1, 0:2]),1)
    # inp_std = inp_no_start.std(axis=(0, 1))
    # inp_mean = inp_no_start.mean(axis=(0, 1))
    # inp_norm= inp_no_start
    # inp_norm = (inp_no_start - inp_mean) / inp_std

    # vis=inp_te_np[:,1:,2:4]/np.linalg.norm(inp_te_np[:,1:,2:4],2,axis=2)[:,:,np.newaxis]
    # inp_norm=np.concatenate((inp_norm,vis),2)
    inp_norm = np.concatenate((inp_te_np, inp_speed), 2)
    inp_mean = np.zeros(4)
    inp_std = np.ones(4)

    return (
        inp_norm[:, :gt_size],
        inp_norm[:, gt_size:],
        {
            "mean": inp_mean,
            "std": inp_std,
            "seq_start": inp_te_np[:, 0:1, :].copy(),
            "frames": frames,
            "peds": ped_ids,
        },
    )


def distance_metrics(gt, preds):
    errors = np.zeros(gt.shape[:-1])
    print("errors shape")
    print(errors.shape)
    print("preds shape")
    print(preds.shape)
    print("gt shape")
    print(gt.shape)
    preds = preds[:, :, :2]
    print("preds shape")
    print(preds.shape)
    for i in range(errors.shape[0]):
        for j in range(errors.shape[1]):
            errors[i, j] = scipy.spatial.distance.euclidean(gt[i, j], preds[i, j])
    return errors.mean(), errors[:, -1].mean(), errors


In [4]:
train_dataset, _ = create_dataset(
            "datasets",
            "fpl",
            0,
            12,
            8,
            delim="\t",
            train=True,
            verbose=True,
        )

ovdje
start loading dataset
validation set size -> 0
001 / 041 - loading GOPR0234U20_imputed.txt
Index(['frame', 'ped', 'x', 'y', 'scale', 'head_x', 'head_y', 'height',
       'l_ear_x', 'l_ear_y', 'l_elbow_x', 'l_elbow_y', 'l_eye_x', 'l_eye_y',
       'l_foot_x', 'l_foot_y', 'l_hand_x', 'l_hand_y', 'l_hip_x', 'l_hip_y',
       'l_knee_x', 'l_knee_y', 'l_shoulder_x', 'l_shoulder_y', 'neck_x',
       'neck_y', 'r_ear_x', 'r_ear_y', 'r_elbow_x', 'r_elbow_y', 'r_eye_x',
       'r_eye_y', 'r_foot_x', 'r_foot_y', 'r_hand_x', 'r_hand_y', 'r_hip_x',
       'r_hip_y', 'r_knee_x', 'r_knee_y', 'r_shoulder_x', 'r_shoulder_y',
       'threshold', 'width', 'R_1', 'R_2', 'R_3', 'T_1', 'T_2', 'T_3'],
      dtype='object')
dt columns
Index(['frame', 'ped', 'x', 'y', 'scale', 'head_x', 'head_y', 'height',
       'l_ear_x', 'l_ear_y', 'l_elbow_x', 'l_elbow_y', 'l_eye_x', 'l_eye_y',
       'l_foot_x', 'l_foot_y', 'l_hand_x', 'l_hand_y', 'l_hip_x', 'l_hip_y',
       'l_knee_x', 'l_knee_y', 'l_shoulder_x',

dt columns
Index(['frame', 'ped', 'x', 'y', 'scale', 'head_x', 'head_y', 'height',
       'l_ear_x', 'l_ear_y', 'l_elbow_x', 'l_elbow_y', 'l_eye_x', 'l_eye_y',
       'l_foot_x', 'l_foot_y', 'l_hand_x', 'l_hand_y', 'l_hip_x', 'l_hip_y',
       'l_knee_x', 'l_knee_y', 'l_shoulder_x', 'l_shoulder_y', 'neck_x',
       'neck_y', 'r_ear_x', 'r_ear_y', 'r_elbow_x', 'r_elbow_y', 'r_eye_x',
       'r_eye_y', 'r_foot_x', 'r_foot_y', 'r_hand_x', 'r_hand_y', 'r_hip_x',
       'r_hip_y', 'r_knee_x', 'r_knee_y', 'r_shoulder_x', 'r_shoulder_y',
       'threshold', 'width', 'R_1', 'R_2', 'R_3', 'T_1', 'T_2', 'T_3'],
      dtype='object')
008 / 041 - loading GOPR0250U20_imputed.txt
Index(['frame', 'ped', 'x', 'y', 'scale', 'head_x', 'head_y', 'height',
       'l_ear_x', 'l_ear_y', 'l_elbow_x', 'l_elbow_y', 'l_eye_x', 'l_eye_y',
       'l_foot_x', 'l_foot_y', 'l_hand_x', 'l_hand_y', 'l_hip_x', 'l_hip_y',
       'l_knee_x', 'l_knee_y', 'l_shoulder_x', 'l_shoulder_y', 'neck_x',
       'neck_y', 'r_ear_x'

016 / 041 - loading GOPR0272U20_imputed.txt
Index(['frame', 'ped', 'x', 'y', 'scale', 'head_x', 'head_y', 'height',
       'l_ear_x', 'l_ear_y', 'l_elbow_x', 'l_elbow_y', 'l_eye_x', 'l_eye_y',
       'l_foot_x', 'l_foot_y', 'l_hand_x', 'l_hand_y', 'l_hip_x', 'l_hip_y',
       'l_knee_x', 'l_knee_y', 'l_shoulder_x', 'l_shoulder_y', 'neck_x',
       'neck_y', 'r_ear_x', 'r_ear_y', 'r_elbow_x', 'r_elbow_y', 'r_eye_x',
       'r_eye_y', 'r_foot_x', 'r_foot_y', 'r_hand_x', 'r_hand_y', 'r_hip_x',
       'r_hip_y', 'r_knee_x', 'r_knee_y', 'r_shoulder_x', 'r_shoulder_y',
       'threshold', 'width', 'R_1', 'R_2', 'R_3', 'T_1', 'T_2', 'T_3'],
      dtype='object')
dt columns
Index(['frame', 'ped', 'x', 'y', 'scale', 'head_x', 'head_y', 'height',
       'l_ear_x', 'l_ear_y', 'l_elbow_x', 'l_elbow_y', 'l_eye_x', 'l_eye_y',
       'l_foot_x', 'l_foot_y', 'l_hand_x', 'l_hand_y', 'l_hip_x', 'l_hip_y',
       'l_knee_x', 'l_knee_y', 'l_shoulder_x', 'l_shoulder_y', 'neck_x',
       'neck_y', 'r_ear_x'

Index(['frame', 'ped', 'x', 'y', 'scale', 'head_x', 'head_y', 'height',
       'l_ear_x', 'l_ear_y', 'l_elbow_x', 'l_elbow_y', 'l_eye_x', 'l_eye_y',
       'l_foot_x', 'l_foot_y', 'l_hand_x', 'l_hand_y', 'l_hip_x', 'l_hip_y',
       'l_knee_x', 'l_knee_y', 'l_shoulder_x', 'l_shoulder_y', 'neck_x',
       'neck_y', 'r_ear_x', 'r_ear_y', 'r_elbow_x', 'r_elbow_y', 'r_eye_x',
       'r_eye_y', 'r_foot_x', 'r_foot_y', 'r_hand_x', 'r_hand_y', 'r_hip_x',
       'r_hip_y', 'r_knee_x', 'r_knee_y', 'r_shoulder_x', 'r_shoulder_y',
       'threshold', 'width', 'R_1', 'R_2', 'R_3', 'T_1', 'T_2', 'T_3'],
      dtype='object')
dt columns
Index(['frame', 'ped', 'x', 'y', 'scale', 'head_x', 'head_y', 'height',
       'l_ear_x', 'l_ear_y', 'l_elbow_x', 'l_elbow_y', 'l_eye_x', 'l_eye_y',
       'l_foot_x', 'l_foot_y', 'l_hand_x', 'l_hand_y', 'l_hip_x', 'l_hip_y',
       'l_knee_x', 'l_knee_y', 'l_shoulder_x', 'l_shoulder_y', 'neck_x',
       'neck_y', 'r_ear_x', 'r_ear_y', 'r_elbow_x', 'r_elbow_y', 'r_ey

034 / 041 - loading GOPR0316U20_imputed.txt
Index(['frame', 'ped', 'x', 'y', 'scale', 'head_x', 'head_y', 'height',
       'l_ear_x', 'l_ear_y', 'l_elbow_x', 'l_elbow_y', 'l_eye_x', 'l_eye_y',
       'l_foot_x', 'l_foot_y', 'l_hand_x', 'l_hand_y', 'l_hip_x', 'l_hip_y',
       'l_knee_x', 'l_knee_y', 'l_shoulder_x', 'l_shoulder_y', 'neck_x',
       'neck_y', 'r_ear_x', 'r_ear_y', 'r_elbow_x', 'r_elbow_y', 'r_eye_x',
       'r_eye_y', 'r_foot_x', 'r_foot_y', 'r_hand_x', 'r_hand_y', 'r_hip_x',
       'r_hip_y', 'r_knee_x', 'r_knee_y', 'r_shoulder_x', 'r_shoulder_y',
       'threshold', 'width', 'R_1', 'R_2', 'R_3', 'T_1', 'T_2', 'T_3'],
      dtype='object')
dt columns
Index(['frame', 'ped', 'x', 'y', 'scale', 'head_x', 'head_y', 'height',
       'l_ear_x', 'l_ear_y', 'l_elbow_x', 'l_elbow_y', 'l_eye_x', 'l_eye_y',
       'l_foot_x', 'l_foot_y', 'l_hand_x', 'l_hand_y', 'l_hip_x', 'l_hip_y',
       'l_knee_x', 'l_knee_y', 'l_shoulder_x', 'l_shoulder_y', 'neck_x',
       'neck_y', 'r_ear_x'

dt columns
Index(['frame', 'ped', 'x', 'y', 'scale', 'head_x', 'head_y', 'height',
       'l_ear_x', 'l_ear_y', 'l_elbow_x', 'l_elbow_y', 'l_eye_x', 'l_eye_y',
       'l_foot_x', 'l_foot_y', 'l_hand_x', 'l_hand_y', 'l_hip_x', 'l_hip_y',
       'l_knee_x', 'l_knee_y', 'l_shoulder_x', 'l_shoulder_y', 'neck_x',
       'neck_y', 'r_ear_x', 'r_ear_y', 'r_elbow_x', 'r_elbow_y', 'r_eye_x',
       'r_eye_y', 'r_foot_x', 'r_foot_y', 'r_hand_x', 'r_hand_y', 'r_hip_x',
       'r_hip_y', 'r_knee_x', 'r_knee_y', 'r_shoulder_x', 'r_shoulder_y',
       'threshold', 'width', 'R_1', 'R_2', 'R_3', 'T_1', 'T_2', 'T_3'],
      dtype='object')


In [15]:
train_dataset[0]

{'src': tensor([[ 4.3798e+02,  4.7059e+02,  5.2552e+01,  9.6000e+02,  4.0149e+02,
           4.0932e+02,  3.6753e+02,  4.5358e+02,  0.0000e+00,  0.0000e+00,
           3.8329e+02,  5.5267e+02,  3.6240e+02,  4.7453e+02,  3.8329e+02,
           4.8225e+02,  3.8579e+02,  5.1880e+02,  3.8325e+02,  4.3010e+02,
           4.0411e+02,  4.3014e+02,  0.0000e+00,  0.0000e+00,  4.2236e+02,
           4.1194e+02,  4.4315e+02,  4.5882e+02,  0.0000e+00,  0.0000e+00,
           4.0414e+02,  5.6052e+02,  4.4568e+02,  4.7448e+02,  4.1193e+02,
           4.8232e+02,  4.0666e+02,  5.2652e+02,  4.2746e+02,  4.3020e+02,
           8.0000e-01,  1.2800e+03,  2.0823e-03,  1.5742e-03, -1.0249e-01,
           2.0628e-04,  1.0911e-02,  2.3511e-04,  0.0000e+00,  0.0000e+00],
         [ 4.7965e+02,  4.6665e+02,  4.8516e+01,  9.6000e+02,  4.0668e+02,
           4.0924e+02,  3.8579e+02,  4.5101e+02,  0.0000e+00,  0.0000e+00,
           3.9627e+02,  5.5002e+02,  3.7282e+02,  4.6941e+02,  4.0147e+02,
           4.7451

In [6]:
tr_dl = torch.utils.data.DataLoader(
        train_dataset, batch_size=100, shuffle=True, num_workers=0
    )

In [7]:
mean = torch.cat(
        (train_dataset[:]["src"][:, 1:, 2:50], train_dataset[:]["trg"][:, :, 2:50]), 1
    ).mean((0, 1))
    # std=train_dataset[:]['src'][:,1:,2:4].std((0,1))
std = torch.cat(
        (train_dataset[:]["src"][:, 1:, 2:50], train_dataset[:]["trg"][:, :, 2:50]), 1
    ).std((0, 1))

In [18]:
mean_list = mean.detach().tolist()
for i, tens in enumerate(mean_list):
    if(tens==0):
        print(tens)

In [23]:
std_list = std.detach().tolist()
for i, tens in enumerate(std_list):
    if(tens==0):
        std_list[i] = 0.00000001
        

In [24]:
std_list

[19.77713394165039,
 1e-08,
 98.3085708618164,
 48.66887664794922,
 109.96125030517578,
 65.9420166015625,
 108.36736297607422,
 65.74494171142578,
 88.2402114868164,
 53.564144134521484,
 152.66001892089844,
 106.49333190917969,
 76.38487243652344,
 27.25556755065918,
 76.02459716796875,
 38.14247131347656,
 84.90787506103516,
 35.82329559326172,
 76.23506927490234,
 23.007404327392578,
 144.5791778564453,
 90.29670715332031,
 118.72766876220703,
 52.55122756958008,
 118.16565704345703,
 67.85592651367188,
 114.02791595458984,
 66.68769073486328,
 97.142333984375,
 61.02817916870117,
 175.38772583007812,
 122.28889465332031,
 89.35462951660156,
 34.443294525146484,
 91.15855407714844,
 44.821407318115234,
 88.18234252929688,
 36.54721450805664,
 1e-08,
 1e-08,
 0.002395414048805833,
 0.0014853960601612926,
 0.020128624513745308,
 0.003571667242795229,
 0.006505582481622696,
 0.003699831198900938,
 17.42995262145996,
 12.738785743713379]

In [8]:
print(mean)

tensor([ 9.1177e+01,  9.6000e+02,  5.4825e+02,  3.6730e+02,  5.2438e+02,
         4.4821e+02,  6.6890e+01,  4.3594e+01,  5.5788e+02,  6.2523e+02,
         4.8907e+02,  4.4518e+02,  5.5856e+02,  5.1063e+02,  5.5797e+02,
         5.7075e+02,  5.4814e+02,  4.1584e+02,  5.7932e+02,  4.2155e+02,
         1.0808e+02,  7.9948e+01,  5.6216e+02,  3.6959e+02,  5.7820e+02,
         4.4156e+02,  4.8270e+01,  4.2115e+01,  5.7561e+02,  6.2408e+02,
         4.9064e+02,  4.0230e+02,  5.9058e+02,  5.1079e+02,  5.8502e+02,
         5.7122e+02,  5.9864e+02,  4.1724e+02,  8.0000e-01,  1.2800e+03,
         1.3163e-03,  7.6087e-04, -1.0251e-01,  2.2396e-04, -2.3791e-04,
         1.1244e-03, -3.3112e-01,  4.2029e-01])


In [9]:
print(std)

tensor([4.8944e+01, 0.0000e+00, 1.9287e+02, 9.1560e+01, 1.8646e+02, 9.4575e+01,
        2.0585e+02, 1.2372e+02, 1.6612e+02, 1.1447e+02, 2.1888e+02, 1.5754e+02,
        1.6482e+02, 5.5518e+01, 1.6039e+02, 8.5171e+01, 1.7271e+02, 5.5466e+01,
        1.6795e+02, 3.6619e+01, 2.3929e+02, 1.6142e+02, 2.0627e+02, 9.2521e+01,
        2.1866e+02, 1.1639e+02, 1.6124e+02, 1.2331e+02, 1.7279e+02, 1.2189e+02,
        2.7152e+02, 1.9167e+02, 1.7047e+02, 6.0338e+01, 1.6417e+02, 8.6169e+01,
        1.8432e+02, 6.0650e+01, 0.0000e+00, 0.0000e+00, 2.6304e-03, 1.6125e-03,
        2.4224e-02, 3.8474e-03, 6.7317e-03, 3.8308e-03, 1.7461e+01, 1.3568e+01])


In [10]:
means = []
stds = []

for i in np.unique(train_dataset[:]["dataset"]):
        ind = train_dataset[:]["dataset"] == i
        means.append(
            torch.cat(
                (
                    train_dataset[:]["src"][ind, 1:, 2:50],
                    train_dataset[:]["trg"][ind, :, 2:50],
                ),
                1,
            ).mean((0, 1))
        )
        stds.append(
            torch.cat(
                (
                    train_dataset[:]["src"][ind, 1:, 2:50],
                    train_dataset[:]["trg"][ind, :, 2:50],
                ),
                1,
            ).std((0, 1))
        )
        



In [12]:

mean = torch.stack(means).mean(0)
print(mean)

tensor([ 7.7997e+01,  9.6000e+02,  5.5596e+02,  3.5295e+02,  5.6157e+02,
         4.3485e+02,  1.7463e+02,  1.0287e+02,  5.8620e+02,  5.9203e+02,
         5.1932e+02,  4.2409e+02,  5.9081e+02,  4.9410e+02,  5.8982e+02,
         5.4570e+02,  5.8551e+02,  4.1511e+02,  6.0393e+02,  4.1973e+02,
         2.4635e+02,  1.5672e+02,  5.3366e+02,  3.5551e+02,  6.0229e+02,
         4.3959e+02,  1.7471e+02,  1.1867e+02,  5.9295e+02,  5.8933e+02,
         5.4174e+02,  4.1839e+02,  6.0387e+02,  4.9158e+02,  5.9928e+02,
         5.4234e+02,  6.1477e+02,  4.1706e+02,  8.0000e-01,  1.2800e+03,
         9.1900e-04,  8.6084e-04, -9.6962e-02,  2.7280e-04, -7.1035e-04,
         7.6699e-04, -2.1354e+00,  7.4604e-01])


In [13]:

std = torch.stack(stds).mean(0)
print(std)

tensor([1.9777e+01, 0.0000e+00, 9.8309e+01, 4.8669e+01, 1.0996e+02, 6.5942e+01,
        1.0837e+02, 6.5745e+01, 8.8240e+01, 5.3564e+01, 1.5266e+02, 1.0649e+02,
        7.6385e+01, 2.7256e+01, 7.6025e+01, 3.8142e+01, 8.4908e+01, 3.5823e+01,
        7.6235e+01, 2.3007e+01, 1.4458e+02, 9.0297e+01, 1.1873e+02, 5.2551e+01,
        1.1817e+02, 6.7856e+01, 1.1403e+02, 6.6688e+01, 9.7142e+01, 6.1028e+01,
        1.7539e+02, 1.2229e+02, 8.9355e+01, 3.4443e+01, 9.1159e+01, 4.4821e+01,
        8.8182e+01, 3.6547e+01, 0.0000e+00, 0.0000e+00, 2.3954e-03, 1.4854e-03,
        2.0129e-02, 3.5717e-03, 6.5056e-03, 3.6998e-03, 1.7430e+01, 1.2739e+01])


In [15]:
zer = torch.zeros(3, 1 ,2)
one = torch.ones(3, 1, 2)

In [16]:
print(zer)
print(one)

tensor([[[0., 0.]],

        [[0., 0.]],

        [[0., 0.]]])
tensor([[[1., 1.]],

        [[1., 1.]],

        [[1., 1.]]])


In [17]:
c = torch.cat((zer, one), 1)

In [18]:
c

tensor([[[0., 0.],
         [1., 1.]],

        [[0., 0.],
         [1., 1.]],

        [[0., 0.],
         [1., 1.]]])

In [10]:
tr_dl = torch.utils.data.DataLoader(
        train_dataset, batch_size=100, shuffle=True, num_workers=0
    )

In [12]:
val_dataset, _ = create_dataset(
            "datasets",
            "fpl",
            0,
            8,
            12,
            delim="/t",
            train=False,
            verbose=True,
        )

ovdje
start loading dataset
validation set size -> 0
001 / 001 - loading GOPR0329U20_imputed.txt
Index(['frame', 'ped', 'x', 'y', 'scale', 'head_x', 'head_y', 'height',
       'l_ear_x', 'l_ear_y', 'l_elbow_x', 'l_elbow_y', 'l_eye_x', 'l_eye_y',
       'l_foot_x', 'l_foot_y', 'l_hand_x', 'l_hand_y', 'l_hip_x', 'l_hip_y',
       'l_knee_x', 'l_knee_y', 'l_shoulder_x', 'l_shoulder_y', 'neck_x',
       'neck_y', 'r_ear_x', 'r_ear_y', 'r_elbow_x', 'r_elbow_y', 'r_eye_x',
       'r_eye_y', 'r_foot_x', 'r_foot_y', 'r_hand_x', 'r_hand_y', 'r_hip_x',
       'r_hip_y', 'r_knee_x', 'r_knee_y', 'r_shoulder_x', 'r_shoulder_y',
       'threshold', 'width', 'R_1', 'R_2', 'R_3', 'T_1', 'T_2', 'T_3'],
      dtype='object')


C:\Users\alika\anaconda3\envs\transformer-trajectory\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
C:\Users\alika\anaconda3\envs\transformer-trajectory\lib\site-packages\pandas\util\_decorators.py:311: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  return func(*args, **kwargs)


ValueError: could not convert string to float: '10\t0\t975.009\t448.3925\t67.01120205607415\t1053.17\t380.507\t1034.91\t445.697\t1050.6\t383.331\t1050.5\t586.575\t1037.55\t469.214\t1055.83\t471.881\t1042.78\t529.168\t1053.18\t404.015\t1076.65\t404.072\t1050.6\t385.842\t1076.69\t377.964\t1128.78\t445.843\t0.0\t0.0\t1100.13\t573.437\t1131.33\t461.374\t1113.15\t469.365\t1076.65\t529.055\t1113.15\t406.607\t0.00250659999438\t0.00426881061867\t-0.110845699906\t-0.00974517129362\t-0.00940682645887\t0.00345828640275'